In [28]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

# 뉴스 데이터 로드
news_data = pd.read_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\한화에어로스페이스\\한화에어로스페이스_test.csv')  # 뉴스 데이터 파일 경로에 맞게 수정
news_content = news_data['content']

In [29]:
news_content

0       [\n\n\n\n\n저장성 이원추진제 추력기(이미지=(주)한화.)한화그룹이 미래 먹...
1       [\n방산, 화학 4개사 빅딜 6년 만에 마무리전문가들에게 손익계산서를 물어보니\n...
2       [\n2021 대한민국 혁신기업 30IT·플랫폼·미래기술 혁신기업한화솔루션·에어로스...
3       [\n항공기 엔진 부품 수혜 전망두달새 주가 20%넘게 올라\n\n\n\n 올 하반...
4       [\n궤도∙자세 제어 핵심 기술인 '저장성 이원추진제 추력기' 개발 나서 \n\n\...
                              ...                        
1816    [\n\t\t\t  모건스탠리캐피털인터내셔널(MSCI) 한국 지수 구성 종목에 한화...
1817    [\n美허니웰벤처스 등과 GPS 교란방어 장치 생산 인피니돔에 900만 달러 투자한...
1818    [\n美허니웰벤처스 등과 GPS 교란방어 장치 업체인피니돔에 총 900만 달러(약 ...
1819    [\n거제 옥포조선소 방문 후 부산 MADEX 2023 등장"이윤 극대화보다 국가 ...
1820    [\n\t\t\t  7일 오후 부산 해운대에 있는 벡스코. 이날부터 사흘간 열리는 ...
Name: content, Length: 1821, dtype: object

In [30]:
import re
 
def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    return text  


In [31]:
# 댓글들(comments)에서 하나씩 가져와 reply라는 변수에 list로 저장

replies = []
i = 0
for comment in news_content:
  replies.append(text_clean(comment))
  i += 1
  news_data['content'][i-1] = replies[i-1]
print(replies[i-1])

C:\Users\tj\AppData\Local\Temp\ipykernel_10648\3114108941.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][i-1] = replies[i-1]
C:\Users\tj\AppData\Local\Temp\ipykernel_10648\3114108941.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][i-1] = replies[i-1]
C:\Users\tj\AppData\Local\Temp\ipykernel_10648\3114108941.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][i-1] =

  일 오후 부산 해운대에 있는 벡스코 이날부터 사흘간 열리는 국제해양방위산업전시회 마덱스는 대우조선해양의 새 주인인 한화오션과 조선 업계 세계 위인 현대중공업이 자존심 대결을 벌이는 모양새였다 이르면 올 하반기 예정인 미니 이지스급 군함으로 불리는 울산급 배치 호위함 수주를 앞두고서다       일 부산 벡스코에서 개막한 국제해양방위산업전에 참가한 한화오션과 현대중공업 부스 모습 김수민 기자          깜짝 등장한 김동관 그룹과 시너지 기대      현재 해군은 단계에 걸쳐 차기 호위함 건조 사업을 진행하고 있다 기존 울산급 호위함과 동해포항급 초계함을 대체하는 프로젝트다 그 중에서도 울산급 배치 번함이 가장 주목 받는다 한국 해군의 미래 호위함이면서 예상 수주금액이 억원대에 달해서다  이 사업에는 한화오션과 현대중공업이 경쟁하고 있다 한화오션은 한화시스템이 개발한 전투 체계를 기본으로 복합식중저속고속 엔진 추진체계를 통해 수중 방사 소음을 최소화한 게 강점이라고 설명한다 이날 현장을 찾은 김동관 한화그룹 부회장은 한화에어로스페이스한화시스템과 함께 많은 분야에서 시너지를 낼 수 있을 것이란 확신을 갖고 있다고 말했다 김승연 한화그룹 회장의 장남인 김 부회장이 한화오션 공개 행사장에 나타난 것은 이번이 처음이다  한화오션이 개념설계 작업에 참여한 합동화력함도 눈길을 끌었다 합동화력함은 다량의 미사일을 싣고 이동식 해상 미사일 기지 역할을 하는 함정이다        김동관 한화그룹 부회장이 일 부산 벡스코에서 열린  국제해양방위산업전의 한화오션 부스를 방문해 전시된 잠수함을 살펴보며 설명을 듣고 있다 사진 한화그룹         선도함부터 마무리까지 현대중공업      바로 옆에 전시관을 마련한 현대중공업도 수주 의지가 남달랐다 한영석 현대중공업 부회장은 기자들과 만나 배치 번함 수주에 대해 잘하는 회사에 줄 것이라고 본다며 현대중공업은 기술력과 품질 납기에서 누구보다 강점이 있다고 생각한다고 자신했다 이 회사 관계자는 선도함인 번함을 연구개발한 현대중공업

C:\Users\tj\AppData\Local\Temp\ipykernel_10648\3114108941.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][i-1] = replies[i-1]
C:\Users\tj\AppData\Local\Temp\ipykernel_10648\3114108941.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][i-1] = replies[i-1]
C:\Users\tj\AppData\Local\Temp\ipykernel_10648\3114108941.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][i-1] =

In [32]:
news_content

0       저장성 이원추진제 추력기이미지주한화한화그룹이 미래 먹거리로 점찍은 인공위성 사업 확...
1       방산 화학 개사 빅딜 년 만에 마무리전문가들에게 손익계산서를 물어보니충남 서산시 대...
2        대한민국 혁신기업 플랫폼미래기술 혁신기업한화솔루션에어로스페이스미래기술 부문에 이름...
3       항공기 엔진 부품 수혜 전망두달새 주가 넘게 올라 올 하반기 실적 호조 지속 전망에...
4       궤도자세 제어 핵심 기술인 저장성 이원추진제 추력기 개발 나서 저장성이원추진제추력기...
                              ...                        
1816      모건스탠리캐피털인터내셔널 한국 지수 구성 종목에 한화에어로스페이스 등 개 종목이...
1817    美허니웰벤처스 등과  교란방어 장치 생산 인피니돔에 만 달러 투자한화에어로스페이스가...
1818    美허니웰벤처스 등과  교란방어 장치 업체인피니돔에 총 만 달러약 억원 투자한화 기존...
1819    거제 옥포조선소 방문 후 부산   등장이윤 극대화보다 국가 안보국제 정세 기여추가 ...
1820      일 오후 부산 해운대에 있는 벡스코 이날부터 사흘간 열리는 국제해양방위산업전시회...
Name: content, Length: 1821, dtype: object

In [33]:
kkma = Kkma()

def text2sentences(text):
    sentences = kkma.sentences(text)
    for idx in range(0, len(sentences)):
        if len(sentences[idx]) <= 10:
            sentences[idx-1] += (' '+sentences[idx])
            sentences[idx] = ''
    return sentences

In [34]:
i = 0
sentences = []
while i < 1821 :
    sentence = text2sentences(news_content[i])
    sentences.append(sentence)
    i += 1

In [35]:
sentences

[['저장성 이원 추진제 추력기 이미지 주한 화한화 그룹이 미래 먹거리로 점찍은 인공위성 사업 확장에 박차를 가한다 이미 계열 사인 한화시스템과 한화에 어로 스페이스가 등 공격투자를 실시한 데 이어 한화가 인공위성 핵심 부품을 자체 개발하기로 결정했다 일 한화는 지난 일 한국 항공 우주연구원 이하 항 우연과 년까지 억원을 투입해 저장성 이원 추진제 추력 기를 함께 개발해 나간다고 밝혔다 추력 기는 인공위성의 궤도 수정 자세 제어 등을 담당하는 부품이다 위성의 수명과 직결되므로 인공위성의 심장이라고도 불린다 그동안 정지 궤도위성에 적용된 이원 추진제 추력 기는 전량 독일 등 해외 기업 제품에 의존해 왔다 한화는 항우연과 함께 추력 기를 국산화한다는 계획이다 한화가 개발 키로 한 저장성 이원 추진제 추력 기는 연료와 산화제를 각기 다른 탱크에 저장하는 이원화 방식이 적용된다 이 경우 연료량을 조절해 효율성이 높고 많은 연료를 장기간 저장할 수 있다는 장점이 있다 김 승 모 한화 방산부문 대표이사는 민간 주도의 뉴 스페이스 시대를 앞두고 해외에 의존하던 핵심 기술을 국산화한다는 의미가 크다고',
  '밝혔다 한화 그룹은 최근 인공위성 사업 확장에 열을 올리고 있다 김 승 연 한화 그룹 회장은 올 초 신년 사에서 미래 모 빌리티 항공 우주 등 신규 사업에서도 세계를 상대로 미래 성장 기회를 선점해 달라고 주문하기도 했다 한화 그룹의 방산 계열 사인 한화에 어로 스페이스는 올 초 소형 인공위성 기업 쎄트 렉아이 지분 를 인수했으며 김 동관 사장은 쎄트 렉아이 등기 임원에 올라 경영에 참여하고 있다 한화시스템은 이미 지난해 영국의 위성통신 안테나 기술 벤처 기업인 페이저 솔루션의 사업 및 자산 일체를 인수한 바 있다'],
 ['방산 화학 개사 빅딜 년 만에 마무리 전문가들에게 손익 계산서를 물어보니 충남 서산시 대산읍 한화 토탈 내 폴리프로필렌 공장 한화 토탈 제공 한화와 삼성그룹 간 방위산업화학 계열 개사 빅딜은 선택과 집중을 필요로 했던 양사 모두에 득이 됐단 평가 

In [36]:
twitter = Twitter()

def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        # 개행 문자 제거 및 리스트에 추가
        lines = [line.strip() for line in lines]
    return lines

# 파일 경로 설정
file_path = './stopword.txt'

# 파일을 리스트로 변환
stopwords = read_file_to_list(file_path)

# 결과 출력
print(stopwords)

['않다', '되어다', '되다', '하다', '어떻다', '이렇다', '이다', '어제', '매일', '아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에

c:\Users\tj\anaconda3\envs\trading\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [37]:
# stopwords.remove('text')

In [38]:
stopwords

['않다',
 '되어다',
 '되다',
 '하다',
 '어떻다',
 '이렇다',
 '이다',
 '어제',
 '매일',
 '아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼

In [39]:
def get_nouns(sentences):
    nouns = []
    for sentence in sentences:
        if sentence is not '':
            nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                   if noun not in stopwords and len(noun) > 1]))
    return nouns

<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\tj\AppData\Local\Temp\ipykernel_10648\1986530246.py:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [40]:
nouns = get_nouns(sentences)

In [41]:
print(sentences[0])
print()
print(nouns[0])

['저장성 이원 추진제 추력기 이미지 주한 화한화 그룹이 미래 먹거리로 점찍은 인공위성 사업 확장에 박차를 가한다 이미 계열 사인 한화시스템과 한화에 어로 스페이스가 등 공격투자를 실시한 데 이어 한화가 인공위성 핵심 부품을 자체 개발하기로 결정했다 일 한화는 지난 일 한국 항공 우주연구원 이하 항 우연과 년까지 억원을 투입해 저장성 이원 추진제 추력 기를 함께 개발해 나간다고 밝혔다 추력 기는 인공위성의 궤도 수정 자세 제어 등을 담당하는 부품이다 위성의 수명과 직결되므로 인공위성의 심장이라고도 불린다 그동안 정지 궤도위성에 적용된 이원 추진제 추력 기는 전량 독일 등 해외 기업 제품에 의존해 왔다 한화는 항우연과 함께 추력 기를 국산화한다는 계획이다 한화가 개발 키로 한 저장성 이원 추진제 추력 기는 연료와 산화제를 각기 다른 탱크에 저장하는 이원화 방식이 적용된다 이 경우 연료량을 조절해 효율성이 높고 많은 연료를 장기간 저장할 수 있다는 장점이 있다 김 승 모 한화 방산부문 대표이사는 민간 주도의 뉴 스페이스 시대를 앞두고 해외에 의존하던 핵심 기술을 국산화한다는 의미가 크다고', '밝혔다 한화 그룹은 최근 인공위성 사업 확장에 열을 올리고 있다 김 승 연 한화 그룹 회장은 올 초 신년 사에서 미래 모 빌리티 항공 우주 등 신규 사업에서도 세계를 상대로 미래 성장 기회를 선점해 달라고 주문하기도 했다 한화 그룹의 방산 계열 사인 한화에 어로 스페이스는 올 초 소형 인공위성 기업 쎄트 렉아이 지분 를 인수했으며 김 동관 사장은 쎄트 렉아이 등기 임원에 올라 경영에 참여하고 있다 한화시스템은 이미 지난해 영국의 위성통신 안테나 기술 벤처 기업인 페이저 솔루션의 사업 및 자산 일체를 인수한 바 있다']

저장성 이원 추진제 추력 이미지 주한 먹거리 인공위성 확장 박차 계열 사인 격투 실시 인공위성 자체 결정 항공 우주 이하 우연 억원 투입 저장성 이원 추진제 추력 추력 인공위성 궤도 수정 자세 제어 위성 수명 직결 인공위성 심장 그동안 정지 궤도 위성 적용 

In [42]:
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []

In [43]:
def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence
sent_graph = build_sent_graph(nouns)

In [44]:
def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
words_graph, idx2word = build_words_graph(nouns)

In [45]:
def get_ranks(graph, d=0.85):
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0
        link_sum = np.sum(A[:,id])
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1
        
    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A,B)
    return {idx: r[0] for idx, r in enumerate(ranks)}

In [46]:
sent_rank_idx = get_ranks(sent_graph)
sent_rank_idx

{0: 0.561162264627915,
 1: 0.5436910476461946,
 2: 0.5028531255418649,
 3: 0.987414050077266,
 4: 0.6014033305898476,
 5: 0.4025875820827721,
 6: 0.4180666559310236,
 7: 0.9188468235448356,
 8: 0.574405275579119,
 9: 1.0052334043977766,
 10: 0.8401639438435683,
 11: 0.6720470392642065,
 12: 0.3508494311552485,
 13: 1.091899049504177,
 14: 0.9913657444902706,
 15: 0.6982694466945081,
 16: 0.8307014623908227,
 17: 0.481411821660833,
 18: 0.7705456760408415,
 19: 0.662989875930179,
 20: 0.7864167553520799,
 21: 0.7864167553520801,
 22: 0.7816215715502735,
 23: 0.7322107875236095,
 24: 0.5751602623736647,
 25: 0.655410985640605,
 26: 0.6303371302762857,
 27: 1.0481562386694196,
 28: 1.1434694354950565,
 29: 0.6838086291601377,
 30: 0.5987172226524624,
 31: 1.173153686685716,
 32: 0.3789818574377568,
 33: 0.9119045745710618,
 34: 1.143895385199388,
 35: 0.4607063260752263,
 36: 1.2995669832122072,
 37: 0.5465658407376482,
 38: 0.42208343854781477,
 39: 0.9315345962315316,
 40: 0.85320711809

In [47]:
sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k:sent_rank_idx[k], reverse=True)

In [48]:
word_rank_idx = get_ranks(words_graph)
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k:word_rank_idx[k], reverse=True)

In [49]:
def summarize(sent_num=10):
    summary = []
    index = []
    for idx in sorted_sent_rank_idx[:sent_num]:
        index.append(idx)
        
    index.sort()
    
    for idx in index:
        summary.append(sentences[idx])
        
    for text in summary:
        print(text)
        print('\n')

In [50]:
summarize()

['누리 호 성공 이후 대 우 상향 방산 성과에 시 총 년 새 조 증가 북한이 기술적 결함에도 군사 정찰위성 발사를 강행했다 실패를 맛본 배경에는 한국형 발사체 누리 호Ⅱ 의 발사 성공이 큰 역할을 했다는 분석이 지배적이다', '이런 가운데 한국형 발사체 고도화 사업의 체계종합기업으로서 누리 호 엔진 개발에 참여 성공에 큰 기여를 한 한화에 어로 스페이스에 대한 관심이 높아 지고 있다 신사업 항공 우주 부문의 성공적인 출발을 뒷받침하는 지상 방산 부문의 실적 호조 덕분에 한화에 어로 스페이스의 주가는 우 상향 곡선 위에 올라탄 모양새다', '특히 한화 오션을 통해 해양까지 사업 시너지를 확장하며 한국판 록히드마틴으로 거듭날 수 있다는 기대 섞인 평가까지 나온다 일 금융투자업계에 따르면 한화에 어로 스페이스의 주가는 누리 호 차 발사에 성공한 지난달 일 이후 지난 일까지 상승했다 만원으로 장을 마감한 일 종가는 지난 월 일 기록했던 주 신고가 만원에 근접한 수준이다 누리 호 차 발사 성공은 민간이 우주산업을 주도하는 뉴스 페이스 시대로 가는 첫발을 뗐다는 의미가 있다 특히 지난해 월 한국형 발사체 고도화 사업의 체계종합기업으로 선정된 한화에 어로 스페이스는 한국 항공 우주연구원 항 우연으로부터 발사체 설계 기술과 발사 운용 관련 기술까지 이전 받는다 누리 호는 오는 년까지 세 차례 더 우주로 날아오를 예정이다 특히 초소형 위성 체계 검증위성 사업을 수주한 자회사 한화시스템과 시너지 효과도 기대된다 다만 항공 우주 부문은 현실적으로는 한화에 어로 스페이스에 유의미한 실적으로까진 이어지지 않고 있는 상황이다 작년 기준 항공 우주 부문 영업이익은 한화에 어로 스페이스 전체 영업이익의 억원에 불과했고 분기엔 각각 영업 손실 억원 억원을 기록하기도 했다 한 증권업계 관계자는 주가를 결정하는 데 있어서는 향후 사업 분야가 성공할 것이란 기대가 큰 영향을 미친다며 누리 호 차 발사 성공이 투자자들의 투자 심리를 자극한 덕분에 주가가 상승 곡선 위에 올라탄 것이라고 평가했다 

In [51]:
def keywords(word_num=100):
    keywords = []
    index = []
    for idx in sorted_word_rank_idx[:word_num]:
        index.append(idx)
    for idx in index:
        keywords.append(idx2word[idx])
    
    print(keywords)

In [52]:
keywords()

['방산', '항공', '미국', '우주', '삼성', '대한민국', '해양', '중인', '국가', '정보', '판단', '누리', '대기업', '성장', '과정', '먼저', '증권', '협력', '계열', '영화', '계속', '품질', '기도', '전투', '선정', '차세대', '속도', '경쟁', '열사', '이건', '성취', '확보', '운행', '페인트', '대해', '선언', '시대', '최대', '앞서', '현장', '운용', '솔루션', '보유', '가야', '분석', '중심', '세상', '수행', '나라', '요구', '성능', '영어', '아무', '불합격', '실제', '공정', '결과', '현대', '탑재', '구체', '보라', '로봇', '발사', '조선', '역량', '전환', '시너지', '의미', '이야기', '기록', '전쟁', '자주포', '비유', '가능성', '주력', '부문', '뉴스', '확인', '방식', '에너지', '변화', '강조', '차원', '제작', '지속', '기간', '기대', '기반', '환경', '조립', '조억원', '보고서', '크게', '국제', '업계', '가운데', '수천', '우리나라', '절대로', '능력']
